## Cargamos todas las bibliotecas necesarias para ejecutar todos los métodos del jupyter. 

Es buena costumbre cargarlos al principio, asi no se estar cargando en memoria repetidas veces cada paquete

In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_validate

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
#import pydotplus

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import warnings
import matplotlib
warnings.filterwarnings('ignore')

In [3]:
sns.set(style="darkgrid")
plt.rcParams['figure.figsize'] = [30, 10]
plt.rcParams['font.size'] = 2

### Creación de funciones auxiliares

Con este método se aplicará la discretización de una determinada columna en función de los cuantiles.
La lógica es:
    - Si el valor x de la columna _column_ cae por debajo del cuantil q1 entonces x -> [1,0,0,0]
    - Si el valor x de la columna _column_ esta entre q1 y q2 entonces x -> [0,1,0,0]
    - Si el valor x de la columna _column_ esta entre q2 y q3 entonces x -> [0,0,1,0]
    - Si el valor x de la columna _column_ es mayor a q3 entonces x -> [0,0,0,1]

In [4]:
def discretize(df, column):
    q1 = df_final[column].quantile(.25)
    q2 = df_final[column].quantile(.5)
    q3 = df_final[column].quantile(.75)
    df[column+"_in_q1"] = 0
    df[column+"_in_q2"] = 0
    df[column+"_in_q3"] = 0
    df[column+"_in_q4"] = 0
    df[column+"_in_q1"] = list(map(lambda x: 1 if x <=q1 else 0, df[column]))
    df[column+"_in_q2"] = list(map(lambda x: 1 if (q1 < x and x <=q2) else 0, df[column]))
    df[column+"_in_q3"] = list(map(lambda x: 1 if (q2 < x and x <=q3) else 0, df[column]))
    df[column+"_in_q4"] = list(map(lambda x: 1 if x>q3 else 0, df[column]))
    df.drop(column, axis=1, inplace=True)
    return df

Con este metodo determinamos si el objeto que se encuentra en la columna _column_ es igual a un _object name_

In [5]:
def is_object(df,column,object_name):
    df["is_"+str(object_name)] = list(map(lambda x: 1 if x == object_name else 0, df[column]))
    return df
    

Mediante los siguientes dos métodos podemos completar los datos faltantes dentro de un DataFrame. Para ello se necesita especificar:
    - Un DataFrame: df
    - Las columnas que contienen NaNs: null_cols
    - El ratio de completitud que se desea: rate_to_complete
    - El metodo con el que se desea completar los NaNs: method

In [6]:
def find_and_replace_nans(df,col,rate_to_complete):
    if df[col][df[col].isnull()].shape[0]>0:
        index_list = df[col][df[col].isnull()].sample(frac=rate_to_complete,random_state=1).index.tolist()

        if len(index_list)>0:
            moda = df[(~df[col].isnull())][col].mode()[0]
            df.loc[index_list,col] = moda
    return df
    
def complete_nans(df,null_cols,rate_to_complete,method):
    if method=="moda_por_clase":
        for col in null_cols:
            df0 = find_and_replace_nans(df[df.target==0.0],col,rate_to_complete)
            df1 = find_and_replace_nans(df[df.target==1.0],col,rate_to_complete)
            df = pd.concat([df0,df1])
    else:
        for col in null_cols:
            df = find_and_replace_nans(df,col,rate_to_complete)
    
    return df.dropna()

Con el metodo _preprocess_ replicamos todos los pasos necesarios para preprocesar la data original. De esta forma podemos realizar la completitud de los datos pasandole un *rate_to_complete*

In [7]:
def preprocess(df, metodo, rate_to_complete):
    
    # Extraemos las filas de la columna 'model' que tienen valores NaNs y luego los completamos con un valor por defecto
    
    #model_null_indexes = df[df.model.isnull()].index.tolist()
    #df.loc[model_null_indexes,"model"] = "other other"
    
    df['model'] = df['model'].fillna('other other')
    
    # Obtenemos las columnas que tienen al menos un NaN
    cols_in_null = df.columns[df.isnull().sum(axis=0)>0].tolist()
    
    # Los completamos con un rate_to_complete y utilizando un determinado metodo
    df_comp = complete_nans(df,cols_in_null,rate_to_complete,metodo)
    
    # Discretizamos las columnas que ya sabemos que tienen una distribucion dada (Chi, Normal)
    # (Se usan cuantiles para ver en qué segmento caen)
    for column in ["diff_time_days_minnone","diff_time_dayssearch engine hit","COUNT(users_logs)","number_visits_model","diff_time_daysbrand listing","diff_time_daysnone",'viewed product', 'visited site', 'staticpage']:
        df_comp = discretize(df_comp,column)
    
    # De la columna 'model' extraemos marca y modelo del celular visitado o  comprado
    df_comp["marca"] = df_comp.model.str.split(" ",1,expand=True)[0]
    df_comp["modelo"] = df_comp.model.str.split(" ",1,expand=True)[1]
    
    # Determinamos de que marca se trata
    df_comp["is_iphone"] = list(map(lambda x: 1 if x=="iPhone" else 0, df_comp["marca"]))
    df_comp["is_samsung"] = list(map(lambda x: 1 if x=="Samsung" else 0, df_comp["marca"]))
    df_comp["is_motorola"] = list(map(lambda x: 1 if x=="Motorola" else 0, df_comp["marca"]))
    df_comp["is_lg"] = list(map(lambda x: 1 if x=="LG" else 0, df_comp["marca"]))
    df_comp["is_sony"] = list(map(lambda x: 1 if x=="Sony" else 0, df_comp["marca"]))
    df_comp["is_other"] = list(map(lambda x: 1 if (x!="Sony" and x!="LG" and x!="Motorola" and x!="Samsung" and x!="iPhone") else 0, df_comp["marca"]))
    
    # determinamos de que modelo se trata
    pop_models = ['6','5s','6S','Galaxy J5','7','7 Plus','Galaxy S8','Galaxy S7','Galaxy S7 Edge','Galaxy J7 Prime','Moto G2 3G Dual','Galaxy S6 Edge','Galaxy S6 Flat','5c','6S Plus','Galaxy J7','6 Plus','Moto G4 Plus','SE','Galaxy Gran Prime Duos TV','4S','Galaxy S8 Plus','5','Moto G3 4G','Galaxy A5 2017','Moto X Play 4G Dual','Moto G5 Plus','Galaxy A7 2017','Moto X2','Galaxy A5']
    for model in pop_models:
        df_comp = is_object(df_comp,"modelo",model)

    df_comp["is_other_model"] = list(map(lambda x: 1 if x not in pop_models else 0, df_comp["modelo"]))
    
    # Eliminamos las columnas con strings
    df_comp.drop(["model","marca","modelo"],axis=1,inplace=True)
    
    return df_comp




Mediante el método *test_completitud* almacenamos los resultados del 5-fold CV para distintos niveles de completitud: yendo desde un 0% a un 80% de completitud, variando a 5%

In [8]:
def test_completitud(train, test, feature_columns, metodo, best_depth):
    dtree = DecisionTreeClassifier(max_depth=5, criterion='gini') 
    completitud_precision_tamanio = {"completitud":[], "precision":[], "tamanio":[]}
    for p in np.arange(0.0,.85,.05):   
        df_train_p = preprocess(train,metodo,p)
        scores = cross_validate(dtree, # Instancia de árbol a entrenar en cada fold
                            df_train_p[feature_columns].values, # X_train features
                            df_train_p["target"].values, # Y_train targets
                            scoring=scoring, # Pedimos que para fold se ejecute esta lista de scorings
                            return_train_score = True, # Queremos ver los scorings de los training folds
                            return_estimator = True, # Ademas, pedimos que se nos devuelvan los dtrees de entrenados
                            cv=5, # Establecemos la cantidad de particiones para realizar la Validación cruzada
                            n_jobs = -1 # Pedimos que se ejecute en paralelo, utilizando todos los cores
                           )
        best_dtree_index = np.argmax(scores["test_roc_auc"])
        best_btree = scores["estimator"][best_dtree_index]

        y_esperado = test.target.values
        y_pred = best_btree.predict(test[feature_columns].values)

        accuracy = accuracy_score(y_esperado, y_pred)
        tamanio = best_btree.tree_.node_count

        completitud_precision_tamanio["completitud"].append(100.0*p)
        completitud_precision_tamanio["precision"].append(accuracy)
        completitud_precision_tamanio["tamanio"].append(tamanio)
        
    return completitud_precision_tamanio

Cargamos los datasets para construir el dataset final

In [9]:
df_first = pd.read_csv('../../data_aa/first_part_dataset.csv')

df_second = pd.read_csv('../../data_aa/second_part_dataset.csv')

df_thrid = pd.read_csv('../../data_aa/thrid_part_dataset.csv')

In [10]:
df_first = df_first.rename(columns={'person_id': 'person'})

df_first = df_first.drop(columns=['Unnamed: 0'])

In [11]:
df_second.columns.values

array(['Unnamed: 0', 'person', 'diff_time_days_minnone',
       'diff_time_daysnone', 'diff_time_dayscheckout',
       'diff_time_daysconversion', 'diff_time_daysbrand listing',
       'diff_time_dayssearch engine hit', 'diff_time_start_end', 'model',
       'number_visits_model'], dtype=object)

In [12]:
df_second = df_second.drop(columns=['Unnamed: 0'])

In [13]:
df_thrid.columns.values

array(['Unnamed: 0', 'person', 'COUNT(users_logs)', 'avg_time_events'],
      dtype=object)

In [14]:
df_thrid = df_thrid.drop(columns=['Unnamed: 0'])

In [15]:
df_final = pd.merge(df_first, df_second, on='person', how='left')

df_final = pd.merge(df_final, df_thrid, on='person', how='left')

In [16]:
df_final.target.isin([1.0]).shape

(19414,)

### Significado de las columnas:

 ##### - Las columnas 'Android', 'BlackBerry', 'Chrome', 'FreeBSD', 'Linux', 'Mac', 'Tizen', 'Ubuntu', 'Unknown', 'Windows','iOS' indican la cantidad de veces que cada usuario ingreso a la pagina de Trocafone utilizando uno de esos esos sistemas operativos. Por ejemplo, si el valor de 'Android' para un usuario es 2 siginfica que ese usuario entro a la pagina dos veces desde un dispositivo con un sistema operativo Android
 
#####  - Las columnas 'Computer', 'Smartphone', 'Tablet' indican la cantidad de veces que cada usuario ingreso a la pagina de Trocafone utilizando uno de esos dispositivos
 
#####  - Las columnas 'ad campaign hit', 'brand listing', 'checkout', 'conversion', 'generic listing', 'lead','search engine hit', 'searched products', 'staticpage','viewed product', 'visited site' indican la cantidad de veces que cada usuario realizo cada uno de esos eventos
 
#####  - La columna 'diff_time_days_minnone' indica la cantidad de tiempo que paso desde el primer evento que realizo el usuario hasta el 2018/06/01
 
#####  - La columna 'diff_time_daysnone' indica la cantidad de tiempo que paso desde el ultimo evento que el usuario realizo en la pagina hasta el 2018/06/01
 
#####  - La columna 'diff_time_dayscheckout' indica la cantidad de tiempo que paso desde que el usuario realizo el ultimo checkout hasta el 2018/06/01
 
#####  - La columna 'diff_time_daysconversion' indica la cantidad de tiempo que paso desde que el usuario realizo la ultima conversion hasta el 2018/06/01
 
#####  - La columna 'diff_time_daysbrand listing' indica la cantidad de tiempo que paso desde que el usuario realizo la ultima brand listing hasta el 2018/06/01
 
#####  - La columna 'diff_time_dayssearch engine hit' indica la cantidad de tiempo que paso desde que el usuario realizo el ultimo engine hit hasta el 2018/06/01
 
#####  - La columna 'diff_time_start_end' indica la cantidad de tiempo que paso entre el primer evento que realizo el usuario en la pagina y el ultimo
 
#####  - La columna 'model' indica el modelo de celular que el usuario mas veces visito
 
#####  - La columna 'number_visits_model' indica cuantas veces el usuario visito el celular que mas visito
 
#####  - La columna 'COUNT(users_logs)' indica la cantidad de eventos que el usuario realizo en la pagina
 
#####  - La columna 'avg_time_events' indica el tiempo promedio que paso entre los eventos realizados por cada usuario

##### - La columna 'Target' indica si el usuario realizo una conversion en la pagina durante el periodo que va desde la fecha 2018/06/01 hasta la 2018/06/15
 

## 1

### Reduccion de dimensionalidad del dataset
Eliminamos las columnas con muy poca información (desvío casi nulo) o alta corrleación (Ej. Windows ~ Computer)

In [17]:
df_final.drop(["BlackBerry","Chrome","FreeBSD","Linux","Mac","Other","Tizen","Ubuntu","Unknown","Windows","person"],
              axis=1,
              inplace=True)

### Conversion de atributos de numericos a categoricos
Transformamos algunas columnas para que su distribución se ajuste a una Chi cuadrado o Normal. Luego discretizamos por cuantiles

In [18]:
print(len(df_final.columns.values))

df_final.columns.values

28


array(['Android', 'Computer', 'Smartphone', 'Tablet', 'ad campaign hit',
       'brand listing', 'checkout', 'conversion', 'generic listing',
       'iOS', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'target', 'viewed product', 'visited site',
       'diff_time_days_minnone', 'diff_time_daysnone',
       'diff_time_dayscheckout', 'diff_time_daysconversion',
       'diff_time_daysbrand listing', 'diff_time_dayssearch engine hit',
       'diff_time_start_end', 'model', 'number_visits_model',
       'COUNT(users_logs)', 'avg_time_events'], dtype=object)

In [19]:
df_final.head(3)

,Android,Computer,Smartphone,Tablet,ad campaign hit,brand listing,checkout,conversion,generic listing,iOS,...,diff_time_daysnone,diff_time_dayscheckout,diff_time_daysconversion,diff_time_daysbrand listing,diff_time_dayssearch engine hit,diff_time_start_end,model,number_visits_model,COUNT(users_logs),avg_time_events
0,0.0,2.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,...,14.3,14.3,NaN,NaN,NaN,0.2,NaN,NaN,6,0.033449
1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,3.0,3.0,NaN,NaN,3.0,0.0,iPhone 7,3.0,17,0.000448
2,1.0,0.0,1.0,0.0,5.0,0.0,1.0,0.0,4.0,0.0,...,0.3,0.3,NaN,NaN,NaN,0.0,iPhone 6S,2.0,19,0.000415


In [20]:
df_final.loc[:,"diff_time_daysnone"] = np.log10(df_final["diff_time_daysnone"]+1.)
df_final.loc[:,"diff_time_days_minnone"] = np.log10(df_final.diff_time_days_minnone+1.)
df_final.loc[:,"diff_time_dayssearch engine hit"] = np.log10(df_final["diff_time_dayssearch engine hit"]+1.0)
df_final.loc[:,"COUNT(users_logs)"] = np.log10(df_final["COUNT(users_logs)"]+1.)
df_final.loc[:,"number_visits_model"] = np.log10(1.+df_final.number_visits_model)
df_final.loc[:,"diff_time_daysbrand listing"] = np.log10(01.+df_final["diff_time_daysbrand listing"])

df_final.loc[:,'viewed product'] = np.log10(01.+df_final['viewed product'])
df_final.loc[:,'visited site'] = np.log10(01.+df_final['visited site'])
df_final.loc[:,'staticpage'] = np.log10(01.+df_final['staticpage'])

Vamos a transformar las columnas Android, Computer, Smartphone y tablet de tal forma que sus valores van a ser solo 0 o 1

In [21]:
df_final['Android'] = np.where(df_final['Android'] == 0, 0, 1)
df_final['Computer'] = np.where(df_final['Computer'] == 0, 0, 1)
df_final['Smartphone'] = np.where(df_final['Smartphone'] == 0, 0, 1)
df_final['Tablet'] = np.where(df_final['Tablet'] == 0, 0, 1)

In [22]:
df_final['model'].head()

0                         NaN
1                    iPhone 7
2                   iPhone 6S
3    Motorola Moto G2 3G Dual
4      Samsung Galaxy S8 Plus
Name: model, dtype: object

En el siguiente metodo se realiza todo el preprocess de los datos. Ademas se completan todos los datos faltantes utilizando la moda de los datos que si estan completos

In [23]:
df_completo = preprocess(df_final.copy(),"moda",1.)

In [24]:
df_completo.columns.values

array(['Android', 'Computer', 'Smartphone', 'Tablet', 'ad campaign hit',
       'brand listing', 'checkout', 'conversion', 'generic listing',
       'iOS', 'lead', 'search engine hit', 'searched products', 'target',
       'diff_time_dayscheckout', 'diff_time_daysconversion',
       'diff_time_start_end', 'avg_time_events',
       'diff_time_days_minnone_in_q1', 'diff_time_days_minnone_in_q2',
       'diff_time_days_minnone_in_q3', 'diff_time_days_minnone_in_q4',
       'diff_time_dayssearch engine hit_in_q1',
       'diff_time_dayssearch engine hit_in_q2',
       'diff_time_dayssearch engine hit_in_q3',
       'diff_time_dayssearch engine hit_in_q4', 'COUNT(users_logs)_in_q1',
       'COUNT(users_logs)_in_q2', 'COUNT(users_logs)_in_q3',
       'COUNT(users_logs)_in_q4', 'number_visits_model_in_q1',
       'number_visits_model_in_q2', 'number_visits_model_in_q3',
       'number_visits_model_in_q4', 'diff_time_daysbrand listing_in_q1',
       'diff_time_daysbrand listing_in_q2',
       

### Creacion del set de training y del de test
Vamos a seleccionar solo 980 casos que no realizaron una compra para balancear el dataset

In [25]:
df_completo_reduced_1 = df_completo[df_completo['target']==1]
print(len(df_completo_reduced_1.index))

980


In [26]:
df_completo_reduced_0 = df_completo[df_completo['target']==0].sample(n=980, random_state=1)
print(len(df_completo_reduced_0.index))

980


In [27]:
df_train = pd.concat([df_completo_reduced_1, df_completo_reduced_0])
del df_completo_reduced_0, df_completo_reduced_1

Nos quedamos con aquellas columnas que no sean target

In [28]:
feature_columns = list(set(df_train.columns.tolist())-set(["target"]))

Obtenemos el dataset de test y lo eliminamos del dataset de train

In [29]:
df_test = df_train.sample(frac=0.2, random_state=2)
df_train.drop(df_test.index,axis=0,inplace=True)
train_indexes = df_train.index.tolist()

In [30]:
len(df_test)

392

Verificamos el balance de categorías en cada datset

In [31]:
N_train = df_train.shape[0]*1.
N_test = df_test.shape[0]*1.

N_train_1 = df_train.target.sum()
N_train_0 = df_train.shape[0]*1.-N_train_1

N_test_1 = df_test.target.sum()
N_test_0 = df_test.shape[0]*1.-N_test_1

In [32]:
print("Porcentaje de casos positivos en el train: %" + str(100.0*N_train_1/N_train))
print("Porcentaje de casos positivos en el test: %" + str(100.0*N_test_1/N_test))

Porcentaje de casos positivos en el train: %50.255102040816325
Porcentaje de casos positivos en el test: %48.97959183673469


## 2

### 2.1 Utilizando únicamente el dataset de train ejecutamos 5-fold Cross Validation, para un modelo de arbol de decision con max_depth igual a 3 y criterion de Gini

In [33]:
scoring = ['accuracy','roc_auc']

In [34]:
dtree = DecisionTreeClassifier(max_depth=3, criterion='gini') 

In [35]:
scores = cross_validate(dtree, # Instancia de árbol a entrenar en cada fold
                        df_train[feature_columns].values, # X_train features
                        df_train["target"].values, # Y_train targets
                        scoring=scoring, # Pedimos que para fold se ejecute esta lista de scorings
                        return_train_score = True, # Queremos ver los scorings de los training folds
                        return_estimator = True, # Ademas, pedimos que se nos devuelvan los dtrees de entrenados
                        cv=5, # Establecemos la cantidad de particiones para realizar la Validación cruzada
                        n_jobs = -1 # Pedimos que se ejecute en paralelo, utilizando todos los cores
                       )



Cargamos los resultados en un DF de Pandas

In [36]:
cv_5_fold_resultados = pd.DataFrame(scores)
cv_5_fold_resultados.drop(["fit_time","score_time","estimator"],axis=1,inplace=True)

In [37]:
cv_5_fold_resultados

,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc
0,0.780255,0.760766,0.801566,0.829917
1,0.770701,0.762360,0.835382,0.826140
2,0.707006,0.778309,0.762739,0.842407
3,0.763578,0.766534,0.815797,0.833026
4,0.766773,0.764143,0.812388,0.830726


Calculamos los estadisticos de cada experimento, tanto para train como para test

In [38]:
cv_5_fold_resultados.describe()

,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc
count,5.000000,5.000000,5.000000,5.000000
mean,0.757663,0.766423,0.805574,0.832443
std,0.029002,0.006982,0.026881,0.006097
min,0.707006,0.760766,0.762739,0.826140
25%,0.763578,0.762360,0.801566,0.829917
50%,0.766773,0.764143,0.812388,0.830726
75%,0.770701,0.766534,0.815797,0.833026
max,0.780255,0.778309,0.835382,0.842407


Seleccionamos el árbol que mejor accuracy haya tenido durante el train

In [39]:
best_dtree_index = np.argmax(scores["test_accuracy"]*scores["test_roc_auc"])

In [40]:
best_btree = scores["estimator"][best_dtree_index]

### Performance del modelo sobre test

Evaluamos su accuracy sobre el 20% restante del dataset que no formó parte del 5-fold CV

In [41]:
y_esperado = df_test.target.values
y_pred = best_btree.predict(df_test[feature_columns].values)

In [42]:
print("Score validation: " + str(accuracy_score(y_esperado, y_pred)))

Score validation: 0.7882653061224489


In [43]:
# dot_data = StringIO()
# export_graphviz(best_btree, out_file=dot_data,  
#                 filled=True, rounded=True,
#                 special_characters=True)
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())

## 2.2 Evaluamos las mejores combinaciones de depth y metrica

Con *get_performance_models* estudiamos el comportamiento del algoritmo *DecisionTree* bajo distintos niveles de profundidad y criterios de selección de features para un dado nodo (Gini o Information Gain)


 -  criterions = ['gini', 'entropy']
 
-  depths = [3, 6, None]

Para tal fin se va a utilizar un procedimiento de 5 fold cross validation. Se informan los resultados de accuracy y auc_roc para validadcion y training de cada fold

Por ultimo se selecciona la combinacion de hiprparamentros que tuvo la mejor performance

In [44]:
def get_performance_models(dataframe, depths, criterions):
    
    d = {'comb_hiperparamentros': [], 'test_accuracy': [],  'train_accuracy': [], 'test_roc_auc': [], 'train_roc_auc': []}
    
    df = pd.DataFrame(data=d)
    
    for criterion in criterions:
        
        for depth in depths:
            
            dtree = DecisionTreeClassifier(max_depth=depth, criterion=criterion)
            
            scoring = ['accuracy','roc_auc']
    
            scores = cross_validate(dtree, # Instancia de árbol a entrenar en cada fold
                                df_train.drop(columns=['target']).values, # X_train features
                                df_train["target"].values, # Y_train targets
                                scoring=scoring, # Pedimos que para fold se ejecute esta lista de scorings
                                return_train_score = True, # Queremos ver los scorings de los training folds
                                return_estimator = True, # Ademas, pedimos que se nos devuelvan los dtrees de entrenados
                                cv=5, # Establecemos la cantidad de particiones para realizar la Validación cruzada
                                n_jobs = -1 # Pedimos que se ejecute en paralelo, utilizando todos los cores
                               )
        
            cv_5_fold_resultados = pd.DataFrame(scores)
            
            cv_5_fold_resultados.drop(["fit_time","score_time","estimator"],axis=1,inplace=True)
                    
            cv_5_fold_resultados['comb_hiperparamentros'] = [criterion + '_' + str(depth)]*5
            
            df = pd.concat([df, cv_5_fold_resultados])
    
    df.rename(columns={'test_accuracy': 'validation_accuracy', 'test_roc_auc': 'validation_roc_auc'}, inplace=True)
    
    return df

In [45]:
criterions = ['gini', 'entropy']

depths = [3, 6, None]

df_performance = get_performance_models(df_train, depths, criterions)

df_performance

,comb_hiperparamentros,validation_accuracy,validation_roc_auc,train_accuracy,train_roc_auc
0,gini_3,0.780255,0.801566,0.760766,0.829917
1,gini_3,0.770701,0.835382,0.762360,0.826140
2,gini_3,0.707006,0.762739,0.778309,0.842407
3,gini_3,0.763578,0.815797,0.766534,0.833026
4,gini_3,0.766773,0.812388,0.764143,0.830726
0,gini_6,0.748408,0.764809,0.836523,0.908199
1,gini_6,0.773885,0.809437,0.847687,0.912964
2,gini_6,0.726115,0.726935,0.846890,0.905411
3,gini_6,0.776358,0.789441,0.841434,0.909060
4,gini_6,0.757188,0.808182,0.830279,0.906678


Para obtener el modelo que obtuvo la mejor performance promediamos los resultados obtenidos sobre el set de test de cada fold para cada combinacion de hiperparamentros. La mejor combinacion de hiperparamentros es la de Information Gain y altura maxima de 6

In [48]:
d = {'validation_accuracy': ['mean']}

df_per = df_performance.groupby(['comb_hiperparamentros']).agg(d)

df_per.columns = ['_'.join(col) for col in df_per.columns.values]

df_per = df_per.reset_index()

df_per[df_per['validation_accuracy_mean']==df_per['validation_accuracy_mean'].max()]

,comb_hiperparamentros,validation_accuracy_mean
1,entropy_6,0.764042


## 2.3 Volvemos a correr el 5-fold CV pero completado el dataset de desarollo con un dado porcentaje de completitud

Se completan los datos faltantes estableciendo un porcentage entre 0% y 80%, variando de a 5%

In [49]:
df_train_80 = preprocess(df_final.loc[train_indexes].copy(),"moda",.25)

In [50]:
df_final.loc[train_indexes].shape

(1568, 28)

In [51]:
scores = cross_validate(dtree, # Instancia de árbol a entrenar en cada fold
                        df_train_80[feature_columns].values, # X_train features
                        df_train_80["target"].values, # Y_train targets
                        scoring=scoring, # Pedimos que para fold se ejecute esta lista de scorings
                        return_train_score = True, # Queremos ver los scorings de los training folds
                        return_estimator = True, # Ademas, pedimos que se nos devuelvan los dtrees de entrenados
                        cv=5, # Establecemos la cantidad de particiones para realizar la Validación cruzada
                        n_jobs = -1 # Pedimos que se ejecute en paralelo, utilizando todos los cores
                       )

In [ ]:
cv_5_fold_resultados = pd.DataFrame(scores)
cv_5_fold_resultados.drop(["fit_time","score_time","estimator"],axis=1,inplace=True)

In [ ]:
cv_5_fold_resultados

,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc
0,0.682540,0.718367,0.657484,0.815793
1,0.661290,0.764228,0.646486,0.824206
2,0.622951,0.777328,0.687222,0.837719
3,0.688525,0.732794,0.755000,0.822053
4,0.622951,0.740891,0.676111,0.838295


In [ ]:
cv_5_fold_resultados.describe()

,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc
count,5.000000,5.000000,5.000000,5.000000
mean,0.655651,0.746721,0.684461,0.827613
std,0.031520,0.023849,0.042494,0.009981
min,0.622951,0.718367,0.646486,0.815793
25%,0.622951,0.732794,0.657484,0.822053
50%,0.661290,0.740891,0.676111,0.824206
75%,0.682540,0.764228,0.687222,0.837719
max,0.688525,0.777328,0.755000,0.838295


In [ ]:
best_dtree_index = np.argmax(scores["test_roc_auc"])

In [ ]:
best_btree = scores["estimator"][best_dtree_index]

In [ ]:
y_esperado = df_test.target.values
y_pred = best_btree.predict(df_test[feature_columns].values)

In [ ]:
print("Score validation: " + str(accuracy_score(y_esperado, y_pred)))

Score validation: 0.5663265306122449


In [ ]:
# dot_data = StringIO()
# export_graphviz(best_btree, out_file=dot_data,  
#                 filled=True, rounded=True,
#                 special_characters=True)
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())

### b y c) Analisis del tamaño del arbol y el accuracy en funcion del porcentaje de datos completos

In [ ]:
test_moda = test_completitud(train=df_final.loc[train_indexes].copy(),
                 test=df_test.copy(),
                 feature_columns=feature_columns,
                 metodo="moda",
                 best_depth=6
                )

In [ ]:
test_moda_por_clase = test_completitud(train=df_final.loc[train_indexes].copy(),
                 test=df_test.copy(),
                 feature_columns=feature_columns,
                 metodo="moda_por_clase",
                 best_depth=6
                )

En en los dos siguientes graficos se observa como, a medida que aumenta el nivel de completitud de los datos, tanto el tamaño del arbol como el nivel de accuracy mejoran.

In [ ]:
sns.set(font_scale=2)
fig, ax1 = plt.subplots()
ax1.plot(test_moda["completitud"],test_moda["tamanio"], linewidth=3.0)
ax1.set_xlabel('Porcentaje de completitud')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Cantidad de nodos del arbol')
plt.title("Imputacion de datos faltantes por moda")
plt.show()

In [ ]:
sns.set(font_scale=2)
fig, ax1 = plt.subplots()
ax1.plot(test_moda["completitud"],test_moda["precision"], linewidth=3.0)
ax1.set_xlabel('Porcentaje de completitud')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Accuracy')
plt.title("Imputacion de datos faltantes por moda")
plt.show()

En en los dos siguientes graficos se observa un comportamiento similar a los graficos anteriores. Es decir, a medida que aumenta el nivel de completitud de los datos, tanto el tamaño del arbol como el nivel de accuracy mejoran.

Tambien podemos observar que la accuracy sobre el set de test es mucho menor, producto de haber overfiteado el modelo al completar datos faltantes con la moda de clase. Esto se produce porque al hacer eso estamos "filtrando" informacion del test de validacion en el test de training, lo que permite obtener muy buenos resultados en el set de desarrollo. Sin embargo, cuando intentamos predecir con el modelo sobre observaciones que no tienen esas caracteristicas los resultados empeoran notablemente.

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(test_moda_por_clase["completitud"],test_moda_por_clase["tamanio"], linewidth=3.0)
ax1.set_xlabel('Porcentaje de completitud')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Cantidad de nodos del arbol')
plt.title("Imputacion de datos faltantes por moda de clase")
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(test_moda_por_clase["completitud"],test_moda_por_clase["precision"], linewidth=3.0)
ax1.set_xlabel('Porcentaje de completitud')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Accuracy')
plt.title("Imputacion de datos faltantes por moda de clase")
plt.show()

## 2.4 Tolerancia al ruido

### a) y b)

In [ ]:
import random

### Generacion de ruido 
Creamos una funcion que genera un array de un largo determinado con valores 0 y un porcentage de valores random. Los valores random que se generen van a depender de la media de la columna para la cual quiere generarse ruido

In [ ]:
def create_array_noise(elements_in_column, percentage_noise, mean_column):
    
    number_noise = round(elements_in_column*percentage_noise)
    
    mu, sigma = 0, mean_column
    
    noise = np.random.normal(mu, sigma, [number_noise,1])
    
    array_noise = noise.tolist()
    
    array_noise_final = []
    
    for item in array_noise:
        
        array_noise_final.append(item[0])
    
    array_no_noise = [0]*(elements_in_column - number_noise)
    
    noise = array_noise_final + array_no_noise
    
    random.shuffle(noise)
    
    return noise    

Creamos una funcion que, utilizando la funcion anterior, agrega ruido a un % dado de valores del atributo que indiquemos

In [ ]:
def generate_dataset_with_noise(df, name_attribute, percentage_noise):
    
    dataframe = df.copy()
    
    number_observations = len(dataframe[name_attribute].values)
    
    mean_column_attribute = dataframe[name_attribute].mean()
    
    array_noise = create_array_noise(number_observations, percentage_noise, mean_column_attribute)
    
    dataframe[name_attribute] = dataframe[name_attribute] + array_noise
    
    return dataframe

Creamos una funcion que recibe como inputs un dataset de train, otro de test y el nombre de un atributo, que sera sobre el cual se va a generar ruido

In [ ]:
def run_model_familiy_datasets(df_train, df_test, name_attribute):
    
    # Modelo con la mejor combinacion de hiperparametros del punto 2.2
    
    dtree = DecisionTreeClassifier(max_depth=6, criterion='gini')
    
    noise_accuracy_tamanio = {"noise":[], "accuracy":[], "tamanio":[]}
    
    # Niveles de ruido
    
    for noise in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35]:
        
        # Agregamos el ruido al attribute elegido
        
        df_noise = generate_dataset_with_noise(df_train, name_attribute, noise)
        
        scoring = ['accuracy']
        
        scores = cross_validate(dtree, # Instancia de árbol a entrenar en cada fold
                            df_noise.drop(columns=['target']).values, # X_train features
                            df_noise["target"].values, # Y_train targets
                            scoring=scoring, # Pedimos que para fold se ejecute esta lista de scorings
                            return_train_score = True, # Queremos ver los scorings de los training folds
                            return_estimator = True, # Ademas, pedimos que se nos devuelvan los dtrees de entrenados
                            cv=5, # Establecemos la cantidad de particiones para realizar la Validación cruzada
                            n_jobs = -1 # Pedimos que se ejecute en paralelo, utilizando todos los cores
                           )
        
        # Seleccionamos el mejor modelo
        
        best_dtree_index = np.argmax(scores["test_accuracy"])
        best_btree = scores["estimator"][best_dtree_index]

        y_esperado = df_test['target'].values
        y_pred = best_btree.predict(df_test.drop(columns=['target']).values)

        accuracy = accuracy_score(y_esperado, y_pred)
        tamanio = best_btree.tree_.node_count

        noise_accuracy_tamanio["noise"].append(100.0*noise)
        noise_accuracy_tamanio["accuracy"].append(accuracy)
        noise_accuracy_tamanio["tamanio"].append(tamanio)
        
    return noise_accuracy_tamanio
       

#### Vamos a seleccionar el atributo que mas 'importancia' tiene en el modelo ('checkout') y el que menos tiene ('Android') para generarles ruido y observar los resultados. La motivacion es analizar que diferencias arrojan los resultados en ambos casos.

#### Nota: Los modelos de arboles de decision de sklearn tienen un atributo  'feature_importances_'. De acuerdo a la documentacion de esta libreria, lo que hace 'feature_importances_' es asignar un coefiente de importancia mayor cuando mas cerca esta la feature de la raiz del arbol.

In [ ]:
dtree = DecisionTreeClassifier(criterion='entropy', max_depth=6)

dtree.fit(df_train.drop(columns=['target']), df_train['target'])

In [ ]:
feature_importances = dtree.feature_importances_

maxposition = feature_importances.tolist().index(max(feature_importances))

df_train.drop(columns=['target']).columns.values[maxposition]

In [ ]:
minposition = feature_importances.tolist().index(min(feature_importances))

df_train.columns.values[minposition]

### e)

#### Lo que se observa en los resultados es que la accuracy no se ve afectada al agregar ruido al atributo 'checkout' ni a 'Android', a pesar de que la primera es la feature mas 'importante' y la segunda la menos. Pareciese que los resultados del arbol son robustos al ruido. Sin embargo, algo que si podemos mencionar es que la variabilidad del accuracy obtenido para los diferentes niveles de noise es mas alta en el caso de la variable 'checkout' que en el de 'Android'.

#### Lo que si se observa es un aumento del tamaño en el arbol a medida que la cantidad de ruido aumenta, en cualquiera de los ods atributos

In [ ]:
df_results_2_4_checkout = run_model_familiy_datasets(df_train, df_test, 'checkout')

pd.DataFrame(data=df_results_2_4_checkout)

In [ ]:
np.std(pd.DataFrame(data=df_results_2_4_checkout)['accuracy'])

In [ ]:
df_results_2_4_min = run_model_familiy_datasets(df_train, df_test, 'Android')

pd.DataFrame(data=df_results_2_4_min)

In [ ]:
np.std(pd.DataFrame(data=df_results_2_4_min)['accuracy'])

### c) Graficamos el tamaño del arbol (cantidad de nodos) en funcion del ruido

In [ ]:
sns.set(font_scale=2)
fig, ax1 = plt.subplots()
ax1.plot(df_results_2_4_checkout["noise"], df_results_2_4_checkout["tamanio"], linewidth=3.0)
ax1.set_xlabel('Porcentaje de noise')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Cantidad de nodos del arbol')
plt.title("Relacion noise y tamaño del arbol - Checkout")
plt.show()

### d) Graficamos la performance (accuracy) en funcion del ruido

In [ ]:
sns.set(font_scale=2)
fig, ax1 = plt.subplots()
ax1.plot(df_results_2_4_checkout["noise"], df_results_2_4_checkout["accuracy"], linewidth=3.0)
ax1.set_xlabel('Porcentaje de noise')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('Accuracy')
plt.title("Relacion noise y accuracy - Checkout")
plt.show()

## 3 Naive bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

### Entrenamos el modelo con los datos de train y predecimos sobre test. Los resultamos que vamos a obtener son las probabilidades que el modelo asigna para cada clase posible (recordemos que en este caso son solo 2: realizo una compra entre el 01/06/2018 y el 15/06/2018 o no)

In [ ]:
clf = GaussianNB()

scoring = ['accuracy','roc_auc']

scores_nb = cross_validate(clf, # Instancia de árbol a entrenar en cada fold
                        df_train.drop(columns=['target']).values, # X_train features
                        df_train["target"].values, # Y_train targets
                        scoring=scoring, # Pedimos que para fold se ejecute esta lista de scorings
                        return_train_score = True, # Queremos ver los scorings de los training folds
                        return_estimator = True, # Ademas, pedimos que se nos devuelvan los dtrees de entrenados
                        cv=5, # Establecemos la cantidad de particiones para realizar la Validación cruzada
                        n_jobs = -1 # Pedimos que se ejecute en paralelo, utilizando todos los cores
                       )
scores_nb

In [ ]:
cv_5_fold_resultados_naive_bayes = pd.DataFrame(scores_nb)

cv_5_fold_resultados_naive_bayes.drop(["fit_time","score_time","estimator"],axis=1,inplace=True)

cv_5_fold_resultados_naive_bayes

In [ ]:
cv_5_fold_resultados_naive_bayes.describe()

In [ ]:
best_naive_bayes_index = np.argmax(scores_nb["test_accuracy"]*scores_nb["test_roc_auc"])

In [ ]:
best_naive_bayes = scores["estimator"][best_naive_bayes_index]

y_esperado_nb = df_test.target.values

y_pred_nb = best_naive_bayes.predict(df_test[feature_columns].values)

#### Accuracy del modelo naive bayes sobre el dataset de test

In [ ]:
print("Score validation: " + str(accuracy_score(y_esperado_nb, y_pred_nb)))

#### Obtenemos las probabilidades de pertenecia a cada clase que el modelo calcula para cada observacion de test

In [ ]:
def prior_probabilities(model, dataset):
    
    y_pred = model.predict_proba(dataset.drop(columns=['target']))
    
    array_pred_class_0 = []
    
    array_pred_class_1 = []
    
    for prediction in y_pred:
        
        array_pred_class_0.append('{:f}'.format(prediction[0]))
        
        array_pred_class_1.append('{:f}'.format(prediction[1]))
    
    d = {'proba_class_0': array_pred_class_0, 'proba_class_1': array_pred_class_1}
    
    df = pd.DataFrame(data=d)
    
    return df     

In [ ]:
prior_probabilities(best_naive_bayes, df_test).head()

#### Obtenemos las probabilidades condicionales para un atributo del dataset

In [ ]:
def get_conditional_probabilities_attribute(dataframe, name_attribute):
    
    df = dataframe.copy()
    
    df['target_no'] = np.where(df['target'] == 0, 1, 0)
    
    d = {'target': ['sum'], 'target_no': ['sum']}

    df_cond = df.groupby([name_attribute]).agg(d)

    df_cond.columns = ['_'.join(col) for col in df_cond.columns.values]

    df_cond = df_cond.reset_index()

    df_cond['target_yes'] = round(df_cond['target_sum']/df_cond['target_sum'].sum(),3)
    
    df_cond['target_no'] = round(df_cond['target_no_sum']/df_cond['target_no_sum'].sum(),3)
    
    df_cond = df_cond.drop(columns=['target_sum', 'target_no_sum'])
    
    return df_cond

#### Por ejemplo, dado el valor de 1 para target, la probabilidad de que 'Android' tome el valor de 0 es de 0,594 

In [ ]:
df_test_cond_proba = get_conditional_probabilities_attribute(df_train, 'Android')

print(df_test_cond_proba.head())

df_test_cond_proba['target_yes'].sum()

## 4
### Naive Bayes vs Arbol de decision

Vamos a utilizar GridSearch para obtener los hiperparametros que maximizan la performance del modelo de arbol de decision. En el caso de Naive Bayes eso no lo hacemos porque el modelo no tiene hiperparametros que optimizar

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'max_depth':[3,5,6,7,9],
               'criterion': ['gini', 'entropy'],
              'min_samples_leaf': [3,6,9]
             }

dtree = DecisionTreeClassifier()

dtree_best = GridSearchCV(cv=5, n_jobs=-1, estimator=dtree, param_grid = param_grid, scoring='roc_auc')

dtree_best = dtree_best.fit(df_train.drop(columns='target'), df_train['target'])

In [ ]:
print(dtree_best.best_score_)
print(dtree_best.best_params_)

In [ ]:
y_predicted = dtree_best.predict(df_test.drop(columns=['target']))

print("Roc_auc decisition tree: " + str(roc_auc_score(df_test['target'], y_predicted)))

In [ ]:
print("Roc_auc naive bayes: " + str(roc_auc_score(y_esperado_nb, y_pred_nb)))

### Resultados

Se observa que el modelo que mejor performance tiene es el de arbol de decision